# Motor Neuron Segmentation

Motor neuron nuclei are labelled with antibodies against ISL1 and they are actually the cells of interest.
Although it would be nice to segment all nuclei, the primary objectivo is to find the motor neurons.

Let's select an image and then crop out a small piece of it to make testing easier.

In [1]:
from pathlib import Path

import numpy as np
from czifile import CziFile
from napari import Viewer
from scipy.ndimage import distance_transform_edt

In [2]:
DATA_DIR = Path("../data/NS#1_Healhty-DAPI, WGA, ISL1, G3BP.czi")
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[2, :, 1000:1500, 1000:1500]

In [3]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])

## Histogram Equalization

We should first try some histogram equalization as the intensity values ar every different across planes.
I am not sure this will solve some of the saturation problems though.

In [4]:
from skimage.exposure import equalize_adapthist  # noqa

In [9]:
kernel = (50 / scale["Z"], 50 / scale["Y"], 50 / scale["X"])

equalized_img = equalize_adapthist(
    img,
    kernel_size=kernel,
    clip_limit=0.001,
)

In [10]:
viewer = Viewer()
viewer.add_image(img, scale=spacing, visible=False)
viewer.add_image(equalized_img, scale=spacing, visible=True)

<Image layer 'equalized_img' at 0x7041ffcfaea0>

## Cellpose

Let's begin by trying out cellpose.

In [11]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa
from skimage.morphology import remove_small_objects  # noqa

logger_setup()

2024-07-01 13:39:24,622 [INFO] WRITING LOG OUTPUT TO /home/biif/.cellpose/run.log
2024-07-01 13:39:24,637 [INFO] 
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.12.4 
torch version:  	2.3.1+cu121


(<Logger cellpose.io (INFO)>, PosixPath('/home/biif/.cellpose/run.log'))

In [12]:
model = models.Cellpose(model_type="cyto3")

2024-07-01 13:39:24,651 [INFO] >>>> using CPU
2024-07-01 13:39:24,653 [INFO] >> cyto3 << model set to be used
2024-07-01 13:39:24,781 [INFO] >>>> loading model /home/biif/.cellpose/models/cyto3
2024-07-01 13:39:24,922 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


Estimate anisotropy: Z to X ratio

In [13]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

5.505521313330107


In [14]:
diameter = 10 / scale["X"]
print(diameter)

80.96354872544275


Estimated diameter of a cell is 8 to 14 $\mu$m.

In [15]:
masks, flows, styles, diams = model.eval(
    [img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=True,
    anisotropy=anisotropy,
    # flow_threshold=2,
    compute_masks=True,
)

2024-07-01 13:39:25,077 [INFO] channels set to [0, 0]
2024-07-01 13:39:25,079 [INFO] ~~~ FINDING MASKS ~~~
2024-07-01 13:39:25,080 [INFO] multi-stack tiff read in as having 110 planes 1 channels
2024-07-01 13:39:26,821 [INFO] running YX: 110 planes of size (500, 500)
2024-07-01 13:40:02,382 [INFO] 100%|##########| 4/4 [00:35<00:00,  8.88s/it]
2024-07-01 13:40:03,376 [INFO] running ZY: 500 planes of size (110, 500)
2024-07-01 13:44:43,460 [INFO] 100%|##########| 32/32 [04:39<00:00,  8.75s/it]
2024-07-01 13:44:45,581 [INFO] running ZX: 500 planes of size (110, 500)
2024-07-01 13:50:04,442 [INFO] 100%|##########| 32/32 [05:18<00:00,  9.96s/it]
2024-07-01 13:50:06,018 [INFO] network run in 640.75s
2024-07-01 13:50:46,119 [INFO] masks created in 40.10s
2024-07-01 13:50:48,202 [INFO] >>>> TOTAL TIME 683.13 sec


In [16]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

<Labels layer 'Labels' at 0x7041c075a6f0>

These parameters seem to have worked better.
Masks are quite noisy.

In [ ]:
label = remove_small_objects(masks[0], 200)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(label, scale=spacing)

## StarDist

Maybe stardist might work.
Not tested yet.

## Clesperanto Voronoi Otsu

Maybe Clesperanto's Voronoi and Otsu labelling works well here.

In [ ]:
import pyclesperanto_prototype as cle  # noqa

cle.available_device_names()

In [ ]:
input_gpu = cle.push(img[3])
input_gpu

I was unable to make clesperanto work on the laptop.

## Thresholding

Let's go for typical thresholding techniques as nuclei might be separated enough to not need watershed or maybe a simple watershed.

In [ ]:
from skimage.feature import peak_local_max  # noqa
from skimage.filters import (  # noqa
    median,
    scharr,
    threshold_otsu,
    try_all_threshold,
)
from skimage.morphology import ball, binary_erosion, dilation, disk  # noqa
from skimage.segmentation import watershed  # noqa

Let's begin with some median filtering to enhance a bit the edges.
I'm using my own footprint.

And then use Otsu thresolding.

In [ ]:
pixel_resolution = 0.5 / scale["X"]

footprint = np.stack(
    [
        np.zeros_like(disk(pixel_resolution)),
        disk(pixel_resolution),
        np.zeros_like(disk(pixel_resolution)),
    ]
)
midpoint = footprint.shape[1] // 2
footprint[0, midpoint, midpoint] = 1
footprint[2, midpoint, midpoint] = 1

filtered_image = median(img, footprint=footprint)
# filtered_image = gaussian(img, sigma=(0.8, 2, 2))
threshold = threshold_otsu(filtered_image)
nuclei = filtered_image > threshold

In [ ]:
try_all_threshold(img[10])

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_image(filtered_image, scale=spacing, visible=False)
viewer.add_labels(nuclei, scale=spacing)

Let's apply watershed to get a better separation between nuclei.
We will need the seeds.

In [ ]:
footprint = np.stack([np.zeros_like(disk(1)), disk(1), np.zeros_like(disk(1))])

inter = binary_erosion(nuclei, footprint=footprint)

transformed = distance_transform_edt(inter, sampling=spacing)

distance_between_nuclei = 1.7  # in um
pixel_min_distance = np.floor(distance_between_nuclei / scale["X"]).astype(int)

maxima = peak_local_max(transformed, min_distance=pixel_min_distance)

In [ ]:
print(pixel_min_distance, len(maxima))

We need an image for the edges of the nuclei to limit the watershed.

In [ ]:
edges = scharr(filtered_image)

In [ ]:
viewer = Viewer()
viewer.add_image(transformed, scale=spacing)
viewer.add_image(edges, scale=spacing)
viewer.add_points(
    maxima,
    name="points",
    scale=spacing,
    size=40,
    n_dimensional=True,  # points have 3D "extent"
    face_color="red",
)

In [ ]:
markers = np.zeros(img.shape, dtype=np.uint32)
marker_indices = tuple(np.round(maxima).astype(int).T)
markers[marker_indices] = np.arange(len(maxima)) + 1
markers = dilation(markers, ball(5))

segmented = watershed(
    -transformed,
    markers,
    mask=nuclei,
)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(segmented, scale=spacing)

It's not the best and should be improved, but it's not that bad.